In [ ]:
import datetime
import pandas as pd
import numpy as np
import pprint
from faker import Faker
from presidio_evaluator import InputSample
from presidio_evaluator.data_generator import PresidioDataGenerator
from presidio_evaluator.data_generator.faker_extensions import (
    RecordsFaker, 
    IpAddressProvider, 
    NationalityProvider, 
    OrganizationProvider, 
    UsDriverLicenseProvider, 
    AddressProviderNew
)

# Generate fake PII data using Presidio's data generator

Presidio's data generator is based on the [Python Faker tool](https://faker.readthedocs.io/en/master/)
and allows you to generate a synthetic dataset from sentence templates.
It features wrappers for Faker which allows you to sample from existing sources of fake data.

Example templates:

> I live at {{address}}

> You can email me at {{email}}. Thanks, {{first_name}}

> What's your last name? It's {{last_name}}

> Every time I see you falling I get down on my knees and pray


### Simple example
This uses all the default values to generate 10 samples based on three templates

In [ ]:
from presidio_evaluator.data_generator import PresidioDataGenerator

sentence_templates = [
    "My name is {{name}}",
    "Please send it to {{address}}",
    "I just moved to {{city}} from {{country}}"
]


data_generator = PresidioDataGenerator()
fake_records = data_generator.generate_fake_data(
    templates=sentence_templates, n_samples=10
)

fake_records = list(fake_records)

# Print the spans of the first sample
print(fake_records[0].fake)
print(fake_records[0].spans)



## Generate a full dataset

In this example we customize the data generator to:
1. Accept more types of entities (by adding more providers to Faker. see [Faker's documentation](https://faker.readthedocs.io/en/master/index.html#how-to-create-a-provider)
2. Handle records of multiple PII entities per fake person for a more realistic dataset

We then translate the generated entity types to match Presidio's, and save the new dataset in json and CONLL03 formats.

a. Specify parameters:

In [ ]:
number_of_samples = 1000
cur_time = datetime.date.today().strftime("%B_%d_%Y")

output_file = f"../data/generated_size_{number_of_samples}_date_{cur_time}.json"
output_conll = f"../data/generated_size_{number_of_samples}_date_{cur_time}.tsv"

templates_file_path = "../presidio_evaluator/data_generator/raw_data/templates.txt"
fake_name_generator_file = "../presidio_evaluator/data_generator/raw_data/FakeNameGenerator.com_3000.csv"

lower_case_ratio = 0.05

b. Read [FakeNameGenerator](https://www.fakenamegenerator.com/) data (optional, extends the set of fake values)
and create a `RecordsFaker` which returns a fake PII record (with multiple values) instead of one value,
allowing dependencies between values belonging to the same fake person
(e.g. name = Michael Smith with the email michael.smith@gmail.com).

The `fake_name_generator_file` can be downloaded from https://www.fakenamegenerator.com/order.php

> Note that you can create fake records for multiple name sets, allowing you to adapt the fake data to the real data if needed. 

In [ ]:
# Read FakeNameGenerator CSV
fake_name_generator_df = pd.read_csv(fake_name_generator_file)

# Update to match existing templates
PresidioDataGenerator.update_fake_name_generator_df(fake_name_generator_df)


c. Create a Faker object (in this case, a `RecordsFaker`)

In [ ]:
# Create RecordsFaker (extension which handles records instead of independent values) and add additional specific providers
fake = RecordsFaker(fake_name_generator_df, locale="en_US")

d. Add more providers, not part of the original Faker package

In [ ]:
fake.add_provider(IpAddressProvider) # Both Ipv4 and IPv6 IP addresses
fake.add_provider(NationalityProvider) # Read countries + nationalities from file
fake.add_provider(OrganizationProvider) # Read organization names from file
fake.add_provider(UsDriverLicenseProvider) # Read US driver license numbers from file
fake.add_provider(AddressProviderNew)  # Extend the default address formats Faker supports

e. Create the Presidio Data Generator object and add provider aliases if the templates have a different entity name than the Faker object

In [ ]:
# Create Presidio Data Generator
data_generator = PresidioDataGenerator(custom_faker=fake, lower_case_ratio=lower_case_ratio)

# Create entity aliases (e.g. if faker supports "name" but templates contain "person").
data_generator.add_provider_alias(provider_name="name", new_name="person")
data_generator.add_provider_alias(provider_name="credit_card_number", new_name="credit_card")
data_generator.add_provider_alias(provider_name="date_of_birth", new_name="birthday")



f. Generate data

In [ ]:
sentence_templates = PresidioDataGenerator.read_template_file(templates_file_path)
fake_records = data_generator.generate_fake_data(
    templates=sentence_templates, n_samples=number_of_samples
)

fake_records = list(fake_records)
pprint.pprint(fake_records[0])

#### Verify randomness of dataset

In [ ]:
from collections import Counter
count_per_template_id = Counter([sample.template_id for sample in fake_records])


print(f"Total: {sum(count_per_template_id.values())}")
print(f"Mean numbers of records per template: {sum(count_per_template_id.values())/len(count_per_template_id)}")
print(f"Median numbers of records per template: {np.median(list(count_per_template_id.values()))}")
print(f"Std: {np.std(list(count_per_template_id.values()))}")

#### Which entities did we generate?

In [ ]:
count_per_entity = Counter()
for record in fake_records:
    count_per_entity.update(Counter([span.type for span in record.spans]))

count_per_entity


#### Translate tags to Presidio's supported entities (optional)

In [ ]:
translator = {'person': "PERSON",
         'ip_address': "IP_ADDRESS",
         'us_driver_license': "US_DRIVER_LICENSE",
         'organization': "ORGANIZATION",
         'name_female': "PERSON",
         'address': "ADDRESS",
         'country': "LOCATION",
         'credit_card_number': "CREDIT_CARD",
         'city': "LOCATION",
         'street_name': "ADDRESS",
         'building_number': "ADDRESS",
         'name': "PERSON",
         'iban': "IBAN_CODE",
         'last_name': "PERSON",
         'last_name_male': "PERSON",
         'last_name_female': "PERSON",
         'first_name': "PERSON",
         'first_name_male': "PERSON",
         'first_name_female': "PERSON",
         'phone_number': "PHONE_NUMBER",
         'url': "DOMAIN_NAME",
         'ssn': "US_SSN",
         'email': "EMAIL_ADDRESS",
         'date_time': "DATE_TIME",
         'date_of_birth': "DATE_TIME",
         'day_of_week': "DATE_TIME",
         'name_male': "PERSON",
         'prefix_male': "TITLE",
         'prefix_female': "TITLE",
         'prefix': "TITLE",
         'nationality': "LOCATION",
         'first_name_nonbinary': "PERSON",
         'postcode': "ADDRESS",
         'secondary_address': "ADDRESS",
         'company': "ORGANIZATION",
         'job': "TITLE",
         'zipcode': "ADDRESS",
         'state_abbr': "ADDRESS"}

count_per_entity_new = Counter()
for record in fake_records:
    for span in record.spans:
        span.type = translator[span.type]
        count_per_entity_new[span.type] += 1

count_per_entity_new



#### Transform the fake samples to a list of InputSample objects:

In [ ]:
%%time
input_samples = [InputSample.from_faker_spans_result(faker_spans_result=fake_record) for fake_record in fake_records]

#### Save as json

In [ ]:
InputSample.to_json(dataset=input_samples, output_file=output_file)

#### Create a CONLL like data frame

In [ ]:
conll = InputSample.create_conll_dataset(input_samples)

In [ ]:
conll.to_csv(output_conll,sep="\t")

### Next steps

- Evaluate Presidio using this fake data. [Sample]("4_Evaluate_Presidio_Analyzer.ipynb")
- Split to train/test/validation while ensuring sentences originiating from the same template are all on the same subset. [Sample]("3_Split_by_pattern_#.ipynb")
- Conduct a small exploratory data analysis on the generated data. [Sample]("2_PII_EDA.ipynb")

#### Copyright notice:


Data generated for evaluation was created using Fake Name Generator.

Fake Name Generator identities by the [Fake Name Generator](https://www.fakenamegenerator.com/) 
are licensed under a [Creative Commons Attribution-Share Alike 3.0 United States License](http://creativecommons.org/licenses/by-sa/3.0/us/). Fake Name Generator and the Fake Name Generator logo are trademarks of Corban Works, LLC.